In [14]:
import os
import numpy as np
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#os.environ["OMP_NUM_THREADS"] = str(1)
import torch.nn as nn
import torch
torch.set_num_threads(1)
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

import random
SEED = 1
from sklearn.model_selection import StratifiedKFold

from Utils import EarlyStopping, get_fold_of_data
from spdnetwork.optimizers import MixOptimizer
from Models import Contrastive_CB3_SPD_WithOutBire, Contrastive_CB3_SPD_1Bire, Contrastive_CB3_Combined, Contrastive_CB3
from ProstatexDataset import ProstatexDataset
from DatasetManagement import DatasetManagement
from Losses import NTXentLoss, TripletLoss, NTXentLossLp
import json
import copy
import pandas as pd
import joblib
import math
# device = torch.device('cpu')
# device = torch.device('cuda:0')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

cpu = torch.device('cpu')
#'1.3.4' pandas version
def reset_random_seeds():
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(SEED)
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
args = {
                'subsampling_strategy':{
                    'name': 'subsampling_criteria',
                    'percentage': None
                },
                'optimizer':{
                    'kind': 'Mix_RMSprop',
                    'learning_rate': 1e-6,
                    'momentum': 0.6
                },
                'loss_function': {
                        'kind': 'CombinedLoss',
                        'margin': 0.05,
                        'temperature': 0.07,
                        'mining': {
                            'positives': 'easy',
                            'negatives': 'semihard'
                        }
                },
                'number_folds': 5,
                'backbone_name': 'Contrastive_CB3_Combined',
                'data': 'mertash',
                'sequence_embedding_features': 18432, #Para SPD TS 2080, para solo AC: 18432
                'target_shape': (12, 32, 32),
                'batch_size': 32,
                'epochs': 3,
                    }

In [16]:
import torch
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, average_precision_score

# Supongo que tienes funciones definidas como DatasetManagement y Contrastive_CB3_Combined

def calculate_metrics_for_fold(fold_number, device, args):
    x_train, x_validation, y_train, y_validation, ids_train, ids_val, idx = get_fold_of_data((fold_number-1), None)

    val_data = DatasetManagement(x_validation, y_validation, data_augmentation = 'nothing')
    train_data = DatasetManagement(x_train, y_train)
    test_dataset = val_data

    model_path = f'/data/Experiments/models/mertash_contrastive_fold_{fold_number}.pt'
    model = Contrastive_CB3_Combined(device, args['target_shape'], args['sequence_embedding_features'], 'contrastive')
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model.to(device)

    val_loader = torch.utils.data.DataLoader(
        dataset=test_dataset,
        shuffle=False,
        batch_size=args['batch_size'],
        pin_memory=False
    )

    all_labels = []
    all_predictions = []

    with torch.no_grad():
        model.eval()
        model.to(device)
        for inputs, labels in val_loader:
            for j in range(len(inputs)):
                inputs[j] = inputs[j].to(device).type(torch.float)
            labels = labels.to(device)
            
            outputs, neurons = model(inputs)
            
            # Convert neurons to numpy arrays
            neurons = neurons.cpu().numpy()
            labels = labels.cpu().numpy().flatten()
            
            all_labels.extend(labels)
            all_predictions.extend(neurons)

    # Convert all predictions and labels to numpy arrays
    all_labels = np.array(all_labels)
    all_predictions = np.array(all_predictions)

    # Assuming neurons output has two classes, compute metrics
    if all_predictions.shape[1] == 2:
        # Compute ROC AUC and AUC-PR
        auc_roc = roc_auc_score(all_labels, all_predictions[:, 1])
        auc_pr = average_precision_score(all_labels, all_predictions[:, 1])
        
        # Compute Accuracy
        predictions = np.argmax(all_predictions, axis=1)
        accuracy = accuracy_score(all_labels, predictions)
    else:
        raise ValueError("The output of neurons should have shape (batch_size, 2) for binary classification.")

    return auc_roc, auc_pr, accuracy

# Iterate over all 5 folds
folds = [1, 2, 3, 4, 5]
results = {'AUC-ROC': [], 'AUC-PR': [], 'Accuracy': []}

for fold in folds:
    auc_roc, auc_pr, accuracy = calculate_metrics_for_fold(fold, device, args)
    results['AUC-ROC'].append(auc_roc)
    results['AUC-PR'].append(auc_pr)
    results['Accuracy'].append(accuracy)
    print(f"Fold {fold} - AUC-ROC: {auc_roc:.4f}, AUC-PR: {auc_pr:.4f}, Accuracy: {accuracy:.4f}")

# Compute average and standard deviation for all folds
average_auc_roc = np.mean(results['AUC-ROC'])
std_auc_roc = np.std(results['AUC-ROC'])
average_auc_pr = np.mean(results['AUC-PR'])
std_auc_pr = np.std(results['AUC-PR'])
average_accuracy = np.mean(results['Accuracy'])
std_accuracy = np.std(results['Accuracy'])

print(f"\nAverage AUC-ROC: {average_auc_roc:.4f} ± {std_auc_roc:.4f}")
print(f"Average AUC-PR: {average_auc_pr:.4f} ± {std_auc_pr:.4f}")
print(f"Average Accuracy: {average_accuracy:.4f} ± {std_accuracy:.4f}")


^^^^^^^^^^Entrenando con todo el Dataset^^^^^^^^^^
Entrenando sin porcentajes
Fold 1 - AUC-ROC: 0.8870, AUC-PR: 0.7799, Accuracy: 0.8923
^^^^^^^^^^Entrenando con todo el Dataset^^^^^^^^^^
Entrenando sin porcentajes
Fold 2 - AUC-ROC: 0.8767, AUC-PR: 0.6812, Accuracy: 0.8333
^^^^^^^^^^Entrenando con todo el Dataset^^^^^^^^^^
Entrenando sin porcentajes
Fold 3 - AUC-ROC: 0.8241, AUC-PR: 0.6074, Accuracy: 0.8622
^^^^^^^^^^Entrenando con todo el Dataset^^^^^^^^^^
Entrenando sin porcentajes
Fold 4 - AUC-ROC: 0.9491, AUC-PR: 0.8784, Accuracy: 0.9434
^^^^^^^^^^Entrenando con todo el Dataset^^^^^^^^^^
Entrenando sin porcentajes
Fold 5 - AUC-ROC: 0.8494, AUC-PR: 0.6329, Accuracy: 0.8373

Average AUC-ROC: 0.8773 ± 0.0421
Average AUC-PR: 0.7159 ± 0.1004
Average Accuracy: 0.8737 ± 0.0407
